<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Eduardo Moya
- Nombre de alumno 2: Nicolás Ojeda

### **Link de repositorio de GitHub:** [https://github.com/eduardomoyab/MDS7202_1](https://github.com/eduardomoyab/MDS7202_1)

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
%pip install -qq xgboost optuna

Note: you may need to restart the kernel to use updated packages.


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\andmo\AppData\Local\Temp\ipykernel_5404\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [5]:
# Paso 1: Separar los datos en conjuntos de train, validation y test
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=1/3, random_state=42)

# Paso 2: Implementar un FunctionTransformer para extraer el día, mes y año de la variable date
def extract_date_info(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    df = df.drop(columns = ['date'])
    return df

date_transformer = FunctionTransformer(extract_date_info, validate=False)

# Paso 3: Implementar un ColumnTransformer para procesar los datos numéricos y categóricos
numeric_features = ['lat', 'long', 'pop','price']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False), categorical_features)],
        remainder="passthrough",
        verbose_feature_names_out=False)
preprocessor.set_output(transform='pandas') #Lo fijamos como output pandas

# Paso 4: Guardar los pasos anteriores en un Pipeline con DummyRegressor
pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

# Paso 5: Entrenar el pipeline y reportar la métrica mean_absolute_error sobre los datos de validación
# Hacemos el split de X e Y para train, validation y testeo
global X_train, Y_train, X_val, Y_val, X_test, Y_test
X_train = train_data.drop('quantity', axis=1)
Y_train = train_data['quantity']
X_val = val_data.drop('quantity', axis=1)
Y_val = val_data['quantity']
X_test = test_data.drop('quantity', axis=1)
Y_test = test_data['quantity']

#Entrenamos el pipeline DummyRegressor
pipeline_dummy.fit(X_train, Y_train)
val_predictions_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(Y_val, val_predictions_dummy)
#r_2_dummy = r2_score(Y_val, val_predictions_dummy)
print(f'MAE con Dummy Regressor: {mae_dummy}')


MAE con Dummy Regressor: 13413.17673026018


Un MAE alto con un Dummy Regressor indica que el modelo simplemente promedió los resultados y no pudo capturar la variabilidad o los patrones en los datos para hacer predicciones precisas.

In [7]:
# Paso 6: Entrenar el Pipeline con XGBRegressor y reportar el cambio en MAE
pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(seed=42))
])

pipeline_xgb.fit(X_train, Y_train)
val_predictions_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)
#r_2_xgb = r2_score(Y_val, val_predictions_xgb)
print(f'MAE con XGBoost: {mae_xgb}')


MAE con XGBoost: 2390.7086816537158


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?: 

En este caso al implementar el algoritmo XGBRegressor el MAE disminuye considerablemente de un valor del orden de 13000 a uno del orden de 2400. Si bien la métrica es poco mas de 10000 veces menor que con el Dummy Regressor, sigue siendo un valor alto considerando que se trata del contexto de negocios, por ende, pese a haber una mejora significativa, sigue siendo un mal desempeño.

In [8]:
# Paso 7: Guardar ambos modelos en un archivo .pkl
with open('regressor_dummy.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)
with open('regressor_xgb.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)


## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [9]:
#Creamos un nuevo pipeline utilizando XGBRegressor con constraints
pipeline_xgb_inv = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(seed=42,monotone_constraints={'price':-1})) 
])

pipeline_xgb_inv.fit(X_train, Y_train)
val_predictions_xgb_inv = pipeline_xgb_inv.predict(X_val)
mae_xgb_inv = mean_absolute_error(Y_val, val_predictions_xgb_inv)
#r_2_xgb_inv = r2_score(Y_val, val_predictions_xgb_inv)
print(f'MAE con XGBoost: {mae_xgb_inv}')
# Save the model with constraints to a .pkl file
with open('model_with_constraints.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb_inv, f)

MAE con XGBoost: 2529.758637824583


En este caso se puede observar que la métrica MAE aumentó en comparación con el XGBRegressor sin la relación del precio con la demanda, aunque de forma muy ligera. De acuerdo a lo que mencionaba el amigo, esta métrica debería haber disminuído, sin embargo ocurrió lo contrario, lo que hace suponer que la relación que existe entre la variable objetivo de demanda con el atributo precio no es linealmente inversa, por lo que se esta haciendo un supuesto que es falso y con ello empeorando el rendimiento.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

**HIPERPARAMETROS**

- **Learning Rate**: Controla la contribución de cada árbol al modelo y ayuda a prevenir el sobreajuste. Valores más bajos hacen que el modelo avance más lentamente hacia la convergencia, lo que puede mejorar la generalización. El rango de (0.001, 0.1) es común para este hiperparámetro.
  
- **N_estimators**: Número de árboles o estimadores en el modelo. Un mayor número de estimadores puede mejorar la capacidad del modelo para capturar patrones, pero puede llevar a sobreajuste si es muy alto. El rango de (50, 1000) es amplio para explorar diferentes cantidades de árboles.
  
- **Max_depth**: Profundidad máxima de cada árbol. Controla la complejidad del árbol y su capacidad para ajustarse a los datos. El rango de (3, 10) es apropiado para encontrar un equilibrio entre la complejidad del modelo y el riesgo de sobreajuste.
  
- **Max_leaves**: Número máximo de hojas para cada árbol. Limita la complejidad y puede ayudar a evitar el sobreajuste. El rango de (0, 100) es interesante para explorar diferentes cantidades de hojas.
  
- **Min_child_weight**: Controla la cantidad mínima de instancias necesarias en cada hoja del árbol. Ayuda a evitar particiones que contengan pocos datos. El rango de (1, 5) permite explorar diferentes configuraciones.
  
- **Reg_alpha** y **Reg_lambda**: Parámetros de regularización que penalizan los pesos de las variables para evitar el sobreajuste. El rango de (0, 1) es típico para estos hiperparámetros.
  
- **Min_frequency** (de OneHotEncoder): Controla la frecuencia mínima para considerar una categoría. Esto puede reducir la dimensionalidad al agrupar categorías poco frecuentes.

In [10]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import FunctionTransformer
from xgboost import XGBRegressor
import joblib
import pandas as pd
import numpy as np

In [11]:
# Fijar la semilla
np.random.seed(42)
# Define the objective function
def objective(trial):
    # Dividir los datos

    # Definir hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Hiperparámetros para OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Crear el pipeline con los hiperparámetros sugeridos
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), numeric_features),
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, **onehot_params), categorical_features)
            ],
            remainder="passthrough",
            verbose_feature_names_out=False
        )),
        ('regressor', XGBRegressor(seed=42,**xgb_params))
    ])

    # Entrenar el modelo
    pipeline_xgb.fit(X_train, Y_train)

    # Realizar predicciones en el conjunto de validación
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)

    return mae_xgb

In [12]:
# Configurar optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=42)
study_opt = optuna.create_study(direction='minimize', sampler=sampler)

# Ejecutar la optimización
study_opt.optimize(objective, timeout=300, show_progress_bar=True)

# Obtener los mejores hiperparámetros
best_params = study_opt.best_params
best_mae = study_opt.best_value
num_trials = len(study_opt.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

Número de trials: 294
MAE con los mejores hiperparámetros: 2011.3882909370811
Mejores hiperparámetros: {'learning_rate': 0.062183929680474766, 'n_estimators': 798, 'max_depth': 8, 'max_leaves': 79, 'min_child_weight': 3, 'reg_alpha': 0.035506062538097426, 'reg_lambda': 0.04739167014450342, 'min_frequency': 0.03859764340935939}


In [14]:
# Obtener el mejor modelo con los mejores hiperparámetros
params_xgb = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}
best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=best_params['min_frequency']), categorical_features)
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
    )),
    ('regressor', XGBRegressor(**params_xgb))
])

# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train, Y_train)
val_predictions_xgb_opt = best_model.predict(X_val)
mae_xgb_opt = mean_absolute_error(Y_val, val_predictions_xgb_opt)
print(f'MAE con XGBoost Optimizado: {mae_xgb_opt}')
print(f'Número de trials: {num_trials}')
#Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
#Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')
# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized.pkl')

MAE con XGBoost Optimizado: 2011.3882909370811
Número de trials: 294

Mejores hiperparámetros:
learning_rate: 0.062183929680474766
n_estimators: 798
max_depth: 8
max_leaves: 79
min_child_weight: 3
reg_alpha: 0.035506062538097426
reg_lambda: 0.04739167014450342
min_frequency: 0.03859764340935939


['best_model_optimized.pkl']

El resultado de la optimización muestra una mejora en el MAE (Mean Absolute Error) con respecto a la sección anterior. El MAE se redujo a 2011.39 utilizando los mejores hiperparámetros encontrados por Optuna, lo que sugiere una mejor capacidad predictiva del modelo.

Estos cambios pueden deberse a que Optuna explora un espacio de búsqueda más amplio y detallado para encontrar configuraciones de hiperparámetros que se adapten mejor a los datos, logrando así un modelo más preciso y generalizable. Además, la optimización puede encontrar combinaciones de hiperparámetros que el enfoque manual no pudo descubrir.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [24]:
# Define the objective function with Prunning
import warnings
warnings.filterwarnings("ignore")

def objective_prunners(trial):
    # Definir hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'eval_metric': mean_absolute_error,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Hiperparámetros para OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Configurar Prunning
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, observation_key="validation_0-mean_absolute_error")

    # Crear el pipeline con los hiperparámetros sugeridos y Prunning
    preprocessor_opt = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False, **onehot_params), categorical_features),
            ])
    preprocessor_opt.set_output(transform='pandas') #Lo fijamos como output pandas

    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor',preprocessor_opt),
        ('regressor', XGBRegressor(seed=42,**xgb_params))
    ])

    pipeline_procesamiento = Pipeline(steps=[
        ('date_extraction', date_transformer),
        ('preprocessor', preprocessor_opt),
    ])


    # Entrenar el preprocesador con el conjunto de entrenamiento
    pipeline_procesamiento.fit_transform(X_train)

    # Aplicar la misma transformación al conjunto de validación
    X_val_processed = pipeline_procesamiento.transform(X_val)
    #X_val_processed= pipeline_procesamiento.fit_transform(X_val)


    # Entrenar el modelo con Prunning
    pipeline_xgb.fit(X_train, Y_train, regressor__callbacks=[pruning_callback], regressor__eval_set=[(X_val_processed,Y_val),], regressor__verbose=False)


    # Realizar predicciones en el conjunto de validación
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)

    return mae_xgb

# Configurar optuna
sampler = TPESampler(seed=42)
study_opt_pr = optuna.create_study(direction='minimize', sampler=sampler)

# Ejecutar la optimización con Prunning
study_opt_pr.optimize(objective_prunners, 
                    timeout=300,show_progress_bar = True)

# Obtener los mejores hiperparámetros
best_params = study_opt_pr.best_params
best_mae = study_opt_pr.best_value
num_trials = len(study_opt_pr.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

Número de trials: 467
MAE con los mejores hiperparámetros: 1875.9265869795158
Mejores hiperparámetros: {'learning_rate': 0.09777930544082102, 'n_estimators': 785, 'max_depth': 10, 'max_leaves': 55, 'min_child_weight': 5, 'reg_alpha': 0.6670048076374575, 'reg_lambda': 0.2919059834510165, 'min_frequency': 0.07029676571148127}


In [25]:

# Obtener el mejor modelo con los mejores hiperparámetros
params_xgb_prunning = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}


preprocessor_opt = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False, min_frequency=best_params['min_frequency']), categorical_features),
        ])
best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor_opt),
    ('regressor', XGBRegressor(seed=42,**params_xgb_prunning))
])

# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train, Y_train)
val_predictions_xgb_prunning = best_model.predict(X_val)
mae_xgb_prunning = mean_absolute_error(Y_val, val_predictions_xgb_prunning)
print(f'MAE con XGBoost Optimizado y Prunning: {mae_xgb_prunning}')
print(f'Número de trials: {num_trials}')
# Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
# Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')

# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized_with_pruning.pkl')

MAE con XGBoost Optimizado y Prunning: 1875.9265869795158
Número de trials: 467

Mejores hiperparámetros:
learning_rate: 0.09777930544082102
n_estimators: 785
max_depth: 10
max_leaves: 55
min_child_weight: 5
reg_alpha: 0.6670048076374575
reg_lambda: 0.2919059834510165
min_frequency: 0.07029676571148127


['best_model_optimized_with_pruning.pkl']

El *prunning* es una técnica utilizada durante el entrenamiento de modelos de aprendizaje automático para detener la exploración de configuraciones de hiperparámetros que no están mostrando mejoras significativas en las métricas de rendimiento.

Esta técnica se basa en cortar ('podar') las ramas de búsqueda del espacio de hiperparámetros durante la optimización, deteniendo prematuramente aquellos intentos que no estén mejorando significativamente el rendimiento. Esta estrategia acelera la búsqueda, descartando configuraciones que probablemente no mejoren el modelo.

El prunning debería impactar el entrenamiento reduciendo el tiempo de ejecución, ya que detiene aquellas configuraciones que no están mostrando mejoras significativas en la métrica de rendimiento, evitando gastar tiempo valioso en configuraciones poco prometedoras.

En esta ocasión, utilizando el prunning, se obtuvo un MAE de 1875.93, una leve mejora con respecto a la optimización anterior sin prunning (2011.39). Esto sugiere que el prunning fue efectivo para enfocar la búsqueda en configuraciones más prometedoras, lo que condujo a una mejora en la métrica de evaluación.

Los hiperparámetros encontrados son ligeramente diferentes, lo que podría indicar que el prunning permitió explorar más eficientemente el espacio de búsqueda y descubrir combinaciones óptimas que no fueron identificadas en la búsqueda anterior.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [15]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [16]:
# Gráfico de historial de optimización con Optuna
plot_optimization_history(study_opt)

In [17]:
# Gráfico de historial de optimización con Optuna y Prunning
plot_optimization_history(study_opt_pr)

En el registro de la optimización, se aprecia un patrón común en ambos métodos, Optuna estándar y la versión con Pruning, revelando mejoras notables a partir de la iteración 25, seguidas de un progreso más gradual. No obstante, se destaca que la implementación con Pruning exhibe un gráfico más ordenado y claro en comparación con la optimización estándar, indicativo probable del efecto de las podas que realiza el Pruning. Esta técnica, al podar ramas menos prometedoras durante el entrenamiento, optimiza el proceso, resultando en una visualización más eficiente con menos iteraciones y un rendimiento similarmente mejorado en comparación con el método convencional.

In [18]:
#Grafico de coordenadas paralelas con Optuna
plot_parallel_coordinate(study_opt)

In [19]:
#Grafico de coordenadas paralelas con Optuna y Prunning
plot_parallel_coordinate(study_opt_pr)

El gráfico de coordenadas paralelas es una herramienta que facilita la visualización y comparación de múltiples dimensiones o variables simultáneamente. En los resultados de la optimización, se evidencia que el método con Pruning muestra menos líneas o dimensiones en contraste con la optimización estándar, lo que respalda la idea de una mayor eficiencia en la exploración de hiperparámetros.

Las diversas dimensiones en este gráfico representan los hiperparámetros optimizados durante el proceso. La presencia de menos líneas en el enfoque con Pruning sugiere que se logran resultados favorables con un menor número de iteraciones. Esto indica un enfoque más selectivo en la exploración de combinaciones de hiperparámetros, priorizando aquellas con un impacto más significativo en el rendimiento del modelo."


In [20]:
# Gráfico de importancia de hiperparámetros con Optuna
plot_param_importances(study_opt)

In [21]:
#Gráfico de importancia de hiperparámetros con Optuna y Prunning
plot_param_importances(study_opt_pr)

Los gráficos de importancia de hiperparámetros reflejan cómo cada parámetro contribuye al rendimiento del modelo, evaluando su impacto relativo. En el caso de la optimización con Optuna, el atributo de mayor importancia es `min_frequency`, seguido por `max_leaves` y `reg_lambda`. Por otro lado, cuando se emplea el Pruning junto con Optuna, `min_frequency` también se destaca como el más influyente, seguido por `learning_rate` y `reg_alpha`. En ambos casos, `min_frequency` se posiciona como el parámetro más significativo en la optimización, aunque las posiciones relativas de otros hiperparámetros varían entre los métodos de optimización, lo que sugiere que el Pruning puede dar mayor importancia a otros atributos como `learning_rate` y `reg_alpha` en la mejora del modelo.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [26]:
#Creamos una tabla resumen con todos los MAE de los modelos
data = {'MAE': [mae_dummy, mae_xgb, mae_xgb_inv, mae_xgb_opt, mae_xgb_prunning]}
df_mae = pd.DataFrame(data, index =['Dummy Regressor', 'XGBoost', 'XGBoost Constraint', 'XGBoost Optuna', 'XGBoost Optuna + Prunning'])
df_mae

,MAE
Dummy Regressor,13413.176730
XGBoost,2390.708682
XGBoost Constraint,2529.758638
XGBoost Optuna,2011.388291
XGBoost Optuna + Prunning,1875.926587


In [28]:
#Cargamos el modelo de optuna + prunning y predecimos sobre el conjunto test
best_model = joblib.load('best_model_optimized_with_pruning.pkl')
test_predictions_xgb_prunning = best_model.predict(X_test)
mae_xgb_prunning = mean_absolute_error(Y_test, test_predictions_xgb_prunning)
print(f'MAE con XGBoost Optimizado y Prunning: {mae_xgb_prunning}')

MAE con XGBoost Optimizado y Prunning: 1992.1683712299841


El modelo que presenta el mejor rendimiento es aquel entrenado con XGBoost y la combinación de Optuna y Prunning, mostrando el MAE más bajo entre los modelos.

Al predecir sobre el conjunto de test utilizando el modelo de XGBoost optimizado con Optuna y Prunning, se obtiene un MAE de 1992.17. Se observa una ligera diferencia con respecto al MAE obtenido en el conjunto de validación (1875.93), lo cual se puede deber a que los datos del conjunto de test pueden tener características ligeramente diferentes o sesgos que no se reflejaron en el conjunto de validación. Además, los modelos de machine learning a veces tienen dificultades para generalizar perfectamente a nuevos datos, lo que puede causar variaciones en las métricas entre los conjuntos de validación y test.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>